In [1]:
import argparse
from datetime import datetime
import json
import numpy as np
import operator
import os
import pandas as pd
from tqdm import tqdm

In [20]:
market = 'NYSE'
data_path_eod = '/home/kikyo/data/qt/google_finance/'
data_path_attn = '/home/kikyo/data/qt/attention_data_'
hpg = np.load('/home/kikyo/code/hidden-conc-hypg/hg_' + market + '.npy')
data_path_rr = '/home/kikyo/data/qt/rr8_volumn_' + market[:-1] + '/'
date_format = '%Y-%m-%d %H:%M:%S'
tickers = '/home/kikyo/data/qt/' + market + '_tickers_qualify_dr-0.98_min-5_smooth.csv'
tickers = np.genfromtxt(tickers, dtype=str, delimiter='\t', skip_header=False)
begin_time = datetime.strptime('2012-11-19 00:00:00', date_format)
col = ['rr', 'rr5', 'rr10', 'rr20', 'rr30', 'volumn', 'rr-1', 'rr-5', 'rr-30']

In [17]:
np.max(hpg[1])

4066

In [ ]:
# generate rr data
for index, ticker in enumerate(tqdm(tickers)):
    single_EOD0 = np.genfromtxt(os.path.join(data_path_eod, market + ticker + '_30Y.csv'), dtype=str, delimiter=',', skip_header=True)
    for day in range(len(single_EOD0)):
        day_time = datetime.strptime(single_EOD0[day][0].split(' ')[0] + ' 00:00:00', date_format)
        if day_time >= begin_time:
            volumn_data = single_EOD0[day:, -1].astype(np.float64)
            min_volumn = np.min(volumn_data)
            max_volumn = np.max(volumn_data)
            break
    day_index = 0
    single_data = []
    for day in range(len(single_EOD0)):
        day_time = datetime.strptime(single_EOD0[day][0].split(' ')[0] + ' 00:00:00', date_format)
        if day_time >= begin_time and day + 30 < len(single_EOD0):
            data_dict = {}
            single_EOD = single_EOD0[:, 1:].astype(np.float64)
            days_closeP = [single_EOD[day][4], single_EOD[day - 1][4], single_EOD[day - 5][4], 
                         single_EOD[day - 10][4], single_EOD[day - 20][4], single_EOD[day - 30][4], 
                         single_EOD[day + 1][4], single_EOD[day + 5][4], single_EOD[day + 30][4], 
                         single_EOD[day + 1][-1]]
            data_dict['rr'] = (days_closeP[0] - days_closeP[1]) / days_closeP[1] if days_closeP[1] != 0 else -1234
            data_dict['rr5'] = (days_closeP[0] - days_closeP[2]) / days_closeP[2] if days_closeP[2] != 0 else -1234
            data_dict['rr10'] = (days_closeP[0] - days_closeP[3]) / days_closeP[3] if days_closeP[3] != 0 else -1234
            data_dict['rr20'] = (days_closeP[0] - days_closeP[4]) / days_closeP[4] if days_closeP[4] != 0 else -1234
            data_dict['rr30'] = (days_closeP[0] - days_closeP[5]) / days_closeP[5] if days_closeP[5] != 0 else -1234
            data_dict['rr-1'] = (days_closeP[0] - days_closeP[6]) / days_closeP[6] if days_closeP[6] != 0 else -1234
            data_dict['rr-5'] = (days_closeP[0] - days_closeP[7]) / days_closeP[7] if days_closeP[7] != 0 else -1234
            data_dict['rr-30'] = (days_closeP[0] - days_closeP[8]) / days_closeP[8] if days_closeP[8] != 0 else -1234
            data_dict['volumn'] = (days_closeP[9] - min_volumn) / max_volumn if days_closeP[9] != 0 else -1234
            single_data.append(data_dict)
    single_data = pd.DataFrame(single_data)
    single_data.to_csv(data_path_rr + market + ticker + '.csv')

In [ ]:
for index, ticker in enumerate(tqdm(tickers)):
    single_EOD0 = np.genfromtxt(os.path.join(data_path_eod, market + '_' + ticker + '_30Y.csv'), dtype=str, delimiter=',', skip_header=True)
    single_data = []
    for day in range(len(single_EOD0)):
        day_time = datetime.strptime(single_EOD0[day][0].split(' ')[0] + ' 00:00:00', date_format)
        if day_time >= begin_time and day + 30 < len(single_EOD0):
            data_dict = {}
            single_EOD = single_EOD0[:, 1:].astype(np.float64)
            days_weight = single_EOD[day][0] * single_EOD[day][4]
            data_dict['hyper_weight'] = days_weight if days_weight != 0 else -1234
            single_data.append(data_dict)
    single_data = pd.DataFrame(single_data)
    if len(single_data) < 1244:
        row = np.full((1244 - single_data.shape[0], 1), -1234)
        index = range(single_data.shape[0], 1244)
        pd_1234 = pd.DataFrame(row, index=index, columns=['hyper_weight'])
        single_data = pd.concat([single_data, pd_1234])
    elif len(single_data) > 1244:
        single_data = single_data.drop(index=range(1244, single_data.shape[0]))
    single_data.to_csv(data_path_attn + market + '/' + ticker + '.csv')

  2%|▏         | 35/1737 [04:35<4:51:32, 10.28s/it]

In [ ]:
attention_weight = np.zeros((1244, hpg.shape[1]))
for row in tqdm(range(attention_weight.shape[0])):
    for i in range(np.max(hpg[1])):
        hyper_index = np.where(hpg[1] == i)
        tickers_rel = tickers[hpg[0][hyper_index]]
        weight_list = []
        for ticker in tickers_rel:
            data_weight_path = pd.read_csv(data_path_rr + market + '/' + ticker + '.csv', index_col=0)
            amount = data_weight_path['hyper_weight'][row]
            weight_list.append(amount) if amount > -1000 else weight_list.append(0)
        hyper_attn = np.array(weight_list) / sum(weight_list) if sum(weight_list) > 1 else np.array(weight_list)
        attention_weight[row, hyper_index[0][0]:hyper_index[0][0] + len(hyper_index[0])] = hyper_attn
np.save('/home/kikyo/data/qt/PreConceptWeight_' + market + '.npy', attention_weight)

In [39]:
len(single_EOD)

5453